In [1]:
import pandas as pd
from Korpora import Korpora
from konlpy.tag import Okt
# 일본어 tokenize import
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from joblib import dump
from joblib import load

In [2]:
Genre = ['雑学', 'スポーツ', '理系', '芸能', 'アニメゲーム', '文系']

for i in Genre:
    df = pd.read_csv(f"../Data/{i}.csv", usecols =["Q","K1","K2","K3","K4","A"])

    # 각 열을 문자열로 합쳐서 튜플 리스트 생성
    QnA = [(q + ":" + k1 +","+ k2 +","+ k3 +","+ k4, index, a) for index, q, k1, k2, k3, k4, a in zip(df.index, df['Q'], df['K1'], df['K2'], df['K3'], df['K4'], df["A"])]

    data = [(Question, index) for Question, index, Answer in QnA]
    # 텍스트 벡터화와 분류 모델을 포함한 파이프라인 생성
    model = make_pipeline(
        TfidfVectorizer(),
        MultinomialNB()
    )
    # 모델 학습
    X_train = [item[0] for item in data]
    y_train = [item[1] for item in data]
    model.fit(X_train, y_train)
    # 모델 저장
    dump(model, f'../Model/sim_{i}.joblib')

In [3]:
# 모델 로드
model = load(f'../Model/sim_{Genre[5]}.joblib')

In [4]:
# 입력 문장을 예측
input_text = """次のうち、「北方領土」に含まれる島々はどれ2,文系,隠岐諸島,小笠原諸島,歯舞群島,但美群島"""
predicted_label = model.predict([input_text])[0]

# 예측된 레이블 출력
QnA[predicted_label]

('次のうち、「北方領土」に含まれる島々はどれ2:隠岐諸島,小笠原諸島,歯舞群島,但美群島', 2099, '歯舞群島')